In [ ]:
import json, pycm, pandas as pd
from nl_classifier import NaturalLanguageClassifer
from datetime import datetime

In [ ]:
MODEL_NAME = 'gpt-4'
TEMPERATURE = 0.1
RESULTS_FILENAME = 'wd_gpt-4_serialization_plus_description_results.json'

In [ ]:
experiments = json.load(open(RESULTS_FILENAME, 'r'))

In [ ]:
for i, experiment in enumerate(experiments):
    cls = experiment["concept"]
    if "created" in experiment:
        print(f'{i+1:02}: {cls["label"]:30} {cls["id"]} SKIPPING')
    else:
        classifier = NaturalLanguageClassifer(cls["id"], cls["label"], cls["definition"] + " " + cls["serialization"], MODEL_NAME, TEMPERATURE)
        print(f'{i+1:02}: {cls["label"]:30} {cls["id"]}')
        data = experiment["data"]
        for j, entity in enumerate(data):
            if "predicted" in entity:
                print(f'   {j+1:02}: {entity["label"]:30} {entity["id"]:45} SKIPPING')
            else:
                print(f'   {j+1:02}: {entity["label"]:30} {entity["id"]:45} {entity["actual"]:10} {classifier.tokens_used(entity["label"] + " " + entity["description"] + " " + entity["serialization"]):05} tokens ', end=" ")
                classification = classifier.classify(entity["label"], entity["description"] + " " + entity["serialization"])
                entity["predicted"] = classification["predicted"].lower()
                if entity["actual"] != entity["predicted"]:
                    if entity["actual"] == "positive":
                        print("FN")
                    else:
                        print("FP")
                else:
                    print("")
                entity["rationale"] = classification["rationale"]
            df_results = pd.DataFrame.from_records(data)
            cm = pycm.ConfusionMatrix(df_results["actual"].tolist(), df_results["predicted"].tolist(), digit=2, classes=[ 'positive', 'negative' ])
            experiment["confusion_matrix"] = cm.matrix
            experiment["created"] = datetime.now().isoformat()
            json.dump(experiments, open(RESULTS_FILENAME, 'w+'))
